# Play with pptx and "Config"-slides
in order to run the following code you should:
- have package python-pptx installed

In [1]:
from pptx import Presentation
from pptx.util import Inches, Pt, Cm
import re, os, sys
from pathlib import Path

REPORT_DIR = Path(os.getcwd())
#print(os.getcwd(),REPORT_DIR)

TEST_ROOT = REPORT_DIR.parent.parent.parent
if str(TEST_ROOT) not in sys.path:
    sys.path.append(str(TEST_ROOT))

from baseTestRoot import *

import Scriptum # type: ignore
Scriptum.__all__


['ReportDataFile', 'ManagedDocx', 'ManagedPptx']

In [7]:
t = Scriptum.tag.createTag('color name=blubb uuu')
t.puretag, t.args

('color', {'name': 'blubb', 'uuu': None})

In [9]:
template = Scriptum.ManagedPptx("template.pptx")
prs = template.document

In [10]:
template.config

{'colors': {'darkblue': ('rgb', (0, 16, 134)),
  'amazon': ('rgb', (59, 122, 87)),
  'green': ('rgb', (0, 176, 80)),
  'red': ('rgb', (255, 0, 0)),
  'orange': ('rgb', (255, 192, 0)),
  'yellow': ('rgb', (255, 255, 0)),
  'black': ('rgb', (0, 0, 0)),
  'blue': ('rgb', (0, 0, 255)),
  'darkgrey': ('rgb', (60, 60, 60)),
  'cyan': ('rgb', (0, 255, 255)),
  'grey100': ('rgb', (100, 100, 100)),
  'darkturquoise': ('rgb', (0, 206, 209)),
  'grey120': ('rgb', (120, 120, 120)),
  'azure': ('rgb', (0, 127, 255)),
  'grey': ('rgb', (140, 140, 140)),
  'celeste': ('rgb', (178, 255, 255)),
  'grey160': ('rgb', (160, 160, 160)),
  'grey180': ('rgb', (180, 180, 180)),
  'grey200': ('rgb', (200, 200, 200)),
  'lightgrey': ('rgb', (230, 230, 230)),
  'grey240': ('rgb', (240, 240, 240)),
  'grey245': ('rgb', (245, 245, 245)),
  'nearwhite': ('rgb', (250, 250, 250)),
  'white': ('rgb', (255, 255, 255))},
 'paragraphs': {},
 'sizes': {}}

In [11]:
template.config_layouts

{'colors': <pptx.slide.SlideLayout at 0x21648224d70>}

In [13]:
l=template.config_layouts['colors']
s=l.shapes[1]

In [14]:
s.text_frame.paragraphs[0].runs[0].font.color.theme_color


AttributeError: 'GraphicFrame' object has no attribute 'text_frame'

In [ ]:
reportPptx.setFontColor(s.text_frame.paragraphs[0].runs[0],colorname='darkblue')

In [ ]:
from pptx.dml.color import RGBColor
type(RGBColor.from_string('112233'))

pptx.dml.color.RGBColor

In [ ]:
import pptx
type(color.rgb) == pptx.dml.color.RGBColor
type(s.text_frame.paragraphs[0].runs[0].font)

pptx.text.text.Font

In [ ]:
l=template.config_layouts['Tables']

In [ ]:
shape=l.shapes[3]
shape.has_table

True

In [ ]:
tables = {}
# try to catch tables
for shape in l.shapes:
    if not shape.has_table:
        continue
    table = shape.table
    # first cell contains paragraph with table name
    found = reportPptx.getSimpleTag(table.cell(0,0).text.lower(),'\<table name=[a-z0-9]+\>')
    
    if not found:
        continue
    name = found[1]['name']
    pars = {}
    # get font from cell(0,0)
    c = table.cell(0,0)
    tf = c.text_frame
    pars['default'] = reportPptx._extractFontAndDecorators(tf.paragraphs[0])
    pars['default']['alignment'] = tf.paragraphs[0].alignment
    pars['default']['verical_anchor'] = c.vertical_anchor
    
    for i,cell in enumerate(table.iter_cells()):
        if i == 0:
            continue # was already cell(0,0)
        found = reportPptx.getSimpleTag(cell.text.lower(),'\<[a-z0-9=_\s]+\>')
        if not found:
            continue
        tf = cell.text_frame
        pars[found[0]] = reportPptx._extractFontAndDecorators(tf.paragraphs[0])
        pars[found[0]]['alignment'] = tf.paragraphs[0].alignment
        pars[found[0]]['verical_anchor'] = cell.vertical_anchor
        
        tcPr = cell._tc.get_or_add_tcPr()
        styles = {}
        for c in tcPr.getchildren():
            if 'solidFill' in c.tag:
                color = reportPptx._getColorFromSolidFill(c)
                styles['solidFill'] = {'color': color}
            else:
                for tag in [ 'lnL', 'lnT', 'lnB', 'lnR' ]:
                    if tag in c.tag:
                        color = [ reportPptx._getColorFromSolidFill(subc) for subc in c.getchildren() 
                                 if 'solidFill' in subc.tag ]
                        if color and color[0][0] in ('theme','rgb'):
                            color = color[0]
                            width = dict(c.items()).get('w',0)
                            styles[tag] = {'color': color, 'width': width}
        pars[found[0]]['style'] = styles
    
    tables[name] = pars

print(tables)


{u'test': {'default': {'sz': 10, 'b': False, 'i': False, 'typeface': 'Calibri', 'verical_anchor': None, 'alignment': None}, u'cell_even': {'sz': 10, 'style': {}, 'b': False, 'i': False, 'typeface': 'Calibri', 'verical_anchor': None, 'alignment': None}, u'cell_odd': {'sz': 10, 'style': {'lnT': {'color': ('theme', 'bg1'), 'width': '12700'}}, 'b': False, 'i': False, 'typeface': 'Calibri', 'verical_anchor': None, 'alignment': None}, u'head_row': {'sz': 10, 'style': {'lnL': {'color': ('theme', 'bg1'), 'width': '12700'}, 'lnB': {'color': ('theme', 'bg1'), 'width': '12700'}, 'lnT': {'color': ('theme', 'bg1'), 'width': '12700'}, 'lnR': {'color': ('theme', 'bg1'), 'width': '12700'}}, 'b': False, 'i': False, 'typeface': 'Calibri', 'verical_anchor': None, 'alignment': None}}, u'blue': {'default': {'sz': 10, 'b': True, 'i': False, 'typeface': 'Calibri', 'verical_anchor': 3, 'alignment': 2}, u'cell': {'sz': 10, 'style': {'lnL': {'color': ('theme', 'tx1'), 'width': '12700'}, 'lnB': {'color': ('theme

In [ ]:
prs.save('table.pptx')

In [ ]:
cell = l.shapes[1].table.cell(2,0)

In [ ]:
from pptx.enum.dml import MSO_THEME_COLOR, MSO_FILL
for cell in l.shapes[1].table.iter_cells():
    
    #cell.fill.solid()
    #cell.fill.patterned()
    #cell.fill.back_color.rgb
    if cell.fill.type != MSO_FILL.BACKGROUND:
        cell.fill.solid()
        print(cell.text,cell.fill.fore_color.rgb)
        print(dir(cell.fill.fore_color))
        cell.fill.fore_color.rgb = RGBColor.from_string('112233')
        

<table name=blue> 112233
['__class__', '__delattr__', '__dict__', '__doc__', '__format__', '__getattribute__', '__hash__', '__init__', '__module__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_color', '_validate_brightness_value', '_xFill', 'brightness', 'from_colorchoice_parent', 'rgb', 'theme_color', 'type']
<head_row> 112233
['__class__', '__delattr__', '__dict__', '__doc__', '__format__', '__getattribute__', '__hash__', '__init__', '__module__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_color', '_validate_brightness_value', '_xFill', 'brightness', 'from_colorchoice_parent', 'rgb', 'theme_color', 'type']
<tail_row> 112233
['__class__', '__delattr__', '__dict__', '__doc__', '__format__', '__getattribute__', '__hash__', '__init__', '__module__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__'

In [ ]:
txt=cell._tc.get_or_add_txBody()

In [ ]:
firstpar=cell.text_frame.paragraphs[0]._p.r_lst[0].get_or_add_rPr()

In [ ]:
firstpar=p.get_or_add_rPr()

In [ ]:
firstpar.sz/100

10.0

In [ ]:
firstpar.latin.typeface

'Calibri'

In [ ]:
pic._element._nvXxPr.cNvPr.attrib['title']
pic._element.spPr


In [ ]:
pfile= Presentation("Python_Jump-In_2019-JUL_vx.pptx")

In [ ]:
slide=pfile.slides[0]

In [ ]:
for shape in slide.shapes:
    print(type(shape))
    try:
        print(shape._element.xml)
    except:
        pass

<class 'pptx.shapes.group.GroupShape'>
<p:grpSp xmlns:p="http://schemas.openxmlformats.org/presentationml/2006/main" xmlns:a="http://schemas.openxmlformats.org/drawingml/2006/main" xmlns:r="http://schemas.openxmlformats.org/officeDocument/2006/relationships">
  <p:nvGrpSpPr>
    <p:cNvPr id="30" name="Group 29"/>
    <p:cNvGrpSpPr/>
    <p:nvPr/>
  </p:nvGrpSpPr>
  <p:grpSpPr>
    <a:xfrm>
      <a:off x="6239934" y="5081737"/>
      <a:ext cx="459857" cy="428122"/>
      <a:chOff x="10436772" y="2711669"/>
      <a:chExt cx="714704" cy="561102"/>
    </a:xfrm>
  </p:grpSpPr>
  <p:sp>
    <p:nvSpPr>
      <p:cNvPr id="31" name="Rounded Rectangle 30"/>
      <p:cNvSpPr/>
      <p:nvPr/>
    </p:nvSpPr>
    <p:spPr>
      <a:xfrm>
        <a:off x="10436772" y="2711669"/>
        <a:ext cx="714704" cy="561102"/>
      </a:xfrm>
      <a:prstGeom prst="roundRect">
        <a:avLst/>
      </a:prstGeom>
      <a:solidFill>
        <a:schemeClr val="accent4">
          <a:lumMod val="75000"

In [ ]:
print(shape.element.xml)

<p:grpSp xmlns:p="http://schemas.openxmlformats.org/presentationml/2006/main" xmlns:a="http://schemas.openxmlformats.org/drawingml/2006/main" xmlns:r="http://schemas.openxmlformats.org/officeDocument/2006/relationships">
  <p:nvGrpSpPr>
    <p:cNvPr id="30" name="Group 29"/>
    <p:cNvGrpSpPr/>
    <p:nvPr/>
  </p:nvGrpSpPr>
  <p:grpSpPr>
    <a:xfrm>
      <a:off x="6239934" y="5081737"/>
      <a:ext cx="459857" cy="428122"/>
      <a:chOff x="10436772" y="2711669"/>
      <a:chExt cx="714704" cy="561102"/>
    </a:xfrm>
  </p:grpSpPr>
  <p:sp>
    <p:nvSpPr>
      <p:cNvPr id="31" name="Rounded Rectangle 30"/>
      <p:cNvSpPr/>
      <p:nvPr/>
    </p:nvSpPr>
    <p:spPr>
      <a:xfrm>
        <a:off x="10436772" y="2711669"/>
        <a:ext cx="714704" cy="561102"/>
      </a:xfrm>
      <a:prstGeom prst="roundRect">
        <a:avLst/>
      </a:prstGeom>
      <a:solidFill>
        <a:schemeClr val="accent4">
          <a:lumMod val="75000"/>
        </a:schemeClr>
      </a:sol

In [ ]:
sub1=shape.shapes[1]

In [ ]:
print(sub1.element.xml)

<p:sp xmlns:p="http://schemas.openxmlformats.org/presentationml/2006/main" xmlns:a="http://schemas.openxmlformats.org/drawingml/2006/main" xmlns:r="http://schemas.openxmlformats.org/officeDocument/2006/relationships">
  <p:nvSpPr>
    <p:cNvPr id="32" name="Isosceles Triangle 31">
      <a:hlinkClick r:id="rId2" action="ppaction://program" highlightClick="1"/>
    </p:cNvPr>
    <p:cNvSpPr/>
    <p:nvPr/>
  </p:nvSpPr>
  <p:spPr>
    <a:xfrm rot="5400000">
      <a:off x="10659548" y="2846831"/>
      <a:ext cx="325053" cy="308516"/>
    </a:xfrm>
    <a:prstGeom prst="triangle">
      <a:avLst>
        <a:gd name="adj" fmla="val 45967"/>
      </a:avLst>
    </a:prstGeom>
    <a:solidFill>
      <a:schemeClr val="bg1"/>
    </a:solidFill>
    <a:ln w="9525" cap="rnd"/>
    <a:scene3d>
      <a:camera prst="orthographicFront"/>
      <a:lightRig rig="threePt" dir="t"/>
    </a:scene3d>
    <a:sp3d>
      <a:bevelT prst="relaxedInset"/>
    </a:sp3d>
  </p:spPr>
  <p:style>
    <a:lnRef

In [ ]:

sub1.click_action.hyperlink.address ='file:///c:\\foo\\bar.mp4'

In [ ]:
pfile.save('new.pptx')